In [1]:
import mne
import scipy.io
import numpy as np
import os
import pandas as pd
import queue
import tkinter as tk
import tkinter.ttk as ttk
from tkinter import messagebox

In [2]:
class CrossValidation_Panel:
    def __init__(self,w, Database):
        
        self.w= w
        
        CrossMethod = tk.LabelFrame(self.w, text="Cross Validation Method", height=10, bg='White', width=50, labelanchor='n')
        Rangeframe = tk.LabelFrame(self.w, text="Selected Range", height=10, bg='White', width=50, labelanchor='n')
        Trainframe = tk.LabelFrame(self.w, text="Train", height=10, bg='White', width=50, labelanchor='n')
        Valframe = tk.LabelFrame(self.w, text="Validation", height=10, bg='White', width=50, labelanchor='n')
        Testframe = tk.LabelFrame(self.w, text="Test", height=10, bg='White', width=50, labelanchor='n')
        CrossMethod.grid(row=0, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)
        Rangeframe.grid(row=1, column=0,padx=20,pady=5,  ipady=4,sticky=tk.W)
        Trainframe.grid(row=2, column=0,padx=20,pady=5,  ipady=4,sticky=tk.W)
        Valframe.grid(row=3, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)
        Testframe.grid(row=4, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)

        self.MethodValue = tk.IntVar() 
        
        Fold5Button = tk.Radiobutton(CrossMethod, text='5 fold',variable= self.MethodValue, value=1, bg= 'White', command= self.state) 
        Fold10Button = tk.Radiobutton(CrossMethod, text='10 fold',variable= self.MethodValue, value=2, bg= 'White', command= self.state) 
        LOSOButton = tk.Radiobutton(CrossMethod, text='Leave-One-Subject-Out',variable= self.MethodValue, value=3, bg= 'White', command= self.state) 
        NoneButton = tk.Radiobutton(CrossMethod, text='Self-defined Method',variable= self.MethodValue, value=4, bg= 'White', command= self.state) 
        # CrossVal= ttk.Combobox(CrossMethod,values=['Leave-One-Subject-Out','5 fold','10 fold', 'None' ], state="readonly")
        tk.Label(CrossMethod, text="Individual", bg= 'White').grid(row=0, column=0,padx=20,ipady=2,sticky=tk.W)
        Fold5Button.grid(row=1, column=0,padx=20,ipady=2,sticky=tk.W)
        Fold10Button.grid(row=2, column=0,padx=20,ipady=2,sticky=tk.W)
        tk.Label(CrossMethod, text="Cross-subject", bg= 'White').grid(row=3, column=0,padx=20,ipady=4,sticky=tk.W)
        LOSOButton.grid(row=4, column=0,padx=20,ipady=2,sticky=tk.W)
        NoneButton.grid(row=5, column=0,padx=20,ipady=2,sticky=tk.W)

        tk.Label(Rangeframe, text="Class", bg= 'White').grid(row=0, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Rangeframe, text="Onset Time(ms)", bg= 'White').grid(row=1, column=0,padx=10,ipady=4,sticky=tk.W)
        tk.Label(Trainframe, text="Subject ", bg= 'White').grid(row=0, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Trainframe, text="Session ", bg= 'White').grid(row=1, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Trainframe, text="Split Ratio ", bg= 'White').grid(row=2, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Valframe, text="Subject ", bg= 'White').grid(row=0, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Valframe, text="Session ", bg= 'White').grid(row=1, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Valframe, text="Split Ratio ", bg= 'White').grid(row=2, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Testframe, text="Subject ", bg= 'White').grid(row=0, column=0,padx=20,ipady=4,sticky=tk.W)
        tk.Label(Testframe, text="Session      ", bg= 'White').grid(row=1, column=0,padx=20,ipady=4,sticky=tk.W)

        self.data= Database.data

        self.data.SelectedClass= tk.Entry(Rangeframe) 
        self.data.SelectedClass.insert(0, 'ex: 1,3,4')
        self.data.SelectedOnset= tk.Entry(Rangeframe)
        self.data.SelectedOnset.insert(0, 'ex: -100~3000')
        self.data.TrainSub = tk.Entry(Trainframe)
        self.data.ValSub = tk.Entry(Valframe)
        self.data.TestSub = tk.Entry(Testframe)
        self.data.TrainSess = tk.Entry(Trainframe)
        self.data.ValSess = tk.Entry(Valframe)
        self.data.TestSess = tk.Entry(Testframe)
        self.data.TrainSplit = tk.Entry(Trainframe)
        self.data.ValSplit= tk.Entry(Valframe)

        self.data.SelectedClass.grid(row=0, column=1, padx=20,ipady=4,sticky=tk.W)
        self.data.SelectedOnset.grid(row=1, column=1, padx=20,ipady=4,sticky=tk.W)
        self.data.TrainSub.grid(row=0, column=1, padx=20,ipady=4,sticky=tk.W)
        self.data.ValSub.grid(row=0, column=1, padx=20,ipady=4,sticky=tk.W)
        self.data.TestSub.grid(row=0, column=1, padx=20,ipady=4,sticky=tk.W)

        self.data.TrainSess.grid(row=1, column=1,padx=20,ipady=4,sticky=tk.W)
        self.data.ValSess.grid(row=1, column=1,padx=20,ipady=4,sticky=tk.W)
        self.data.TestSess.grid(row=1, column=1,padx=20,ipady=4,sticky=tk.W)

        self.data.TrainSplit.grid(row=2, column=1,padx=20,ipady=4,sticky=tk.W)
        self.data.ValSplit.grid(row=2, column=1,padx=20,ipady=4,sticky=tk.W)

        
        ttk.Button(w ,text="Confirm", command=(lambda:CrossValidationGroup_Panel(self.w, self.data)), width=10).grid(row=5, column=0, pady=5)

    def state(self):
        self.data.Method= int(self.MethodValue.get())
        if self.data.Method == 3: #LOSO
            self.data.TrainSub['state']= 'disabled'
            self.data.ValSub['state']= 'disabled'
            self.data.TestSub['state']= 'disabled'
            self.data.TrainSess['state']= 'disabled'
            self.data.ValSess['state']= 'disabled'
            self.data.TestSess['state']= 'disabled'
            self.data.TrainSplit['state']= 'disabled'
            self.data.ValSplit['state']= 'disabled'

        elif self.data.Method ==1 or self.data.Method ==2: # 5 or 10 fold 
            self.data.TrainSplit['state']= 'disabled'
            self.data.ValSplit['state']= 'disabled'
        else:
            self.data.TrainSub['state']= 'normal'
            self.data.ValSub['state']= 'normal'
            self.data.TestSub['state']= 'normal'
            self.data.TrainSess['state']= 'normal'
            self.data.ValSess['state']= 'normal'
            self.data.TestSess['state']= 'normal'
            self.data.TrainSplit['state']= 'normal'
            self.data.ValSplit['state']= 'normal'

    
        
        
        

In [3]:
from window import new_window
class CrossValidationGroup_Panel:
    def __init__(self, w, data):
        
        self.w= w
        self.groupw= new_window('Group Data for Training','300x300',0,0,0,0)
        Traingroup = tk.LabelFrame(self.groupw, text="Train", height=10, bg='White', width=50, labelanchor='n')
        Valgroup = tk.LabelFrame(self.groupw, text="Validation", height=10, bg='White', width=50, labelanchor='n')
        Testgroup = tk.LabelFrame(self.groupw, text="Test", height=10, bg='White', width=50, labelanchor='n')
        Traingroup.grid(row=0, column=0,padx=20,pady=5, ipady=4,sticky=tk.W)
        Valgroup.grid(row=1, column=0,padx=20,pady=5,  ipady=4,sticky=tk.W)
        Testgroup.grid(row=2, column=0,padx=20,pady=5,  ipady=4,sticky=tk.W)

        self.data= data
        self.Method= data.Method
        self.SplitbyMethod()
        
    def SplitbyMethod(self):
        
        if self.Method == 3: # loso 
             UseLOSO(self.data, self.groupw) 
        else:
             GetValues(self.w, self.data, self.Method)
        # if self.Method ==1 : 
        #     GetValues(self.w, self.data)
        #     print('individual 5fold')
        # elif self.Method ==2:
        #     GetValues(self.w, self.data)
        #     print('individual 10fold')

        # elif self.Method == 3: # loso 
        #     UseLOSO(self.data, self.groupw) 
              
        # elif self.Method == 4: # selfdefined
        #     GetValues(self.w, self.data)
        #     SplitData(self.groupw,self.data)

In [4]:
class UseLOSO():
    def __init__(self, data, w):
        self.w= w #group data for trianining
        self.AllSub= data.AllSub
        self.q= self.SubOrderInit()
        self.LOSO()
        
        
    def SubOrderInit(self):
        q= queue.Queue()
        [q.put(j) for j in self.AllSub]
        return q
        
    def LOSO(self):
        for group in range(len(self.AllSub)):
            temp= self.q.get() # get first sub
            self.q.put(temp)   # put to the tail
            
            #training process 
           # aftersub.get() # kick out one subject 
            print(group, self.q.queue) # 前四個train 最後一個test 
          
            tk.Label(self.w , text= self.q.queue, bg= 'White').grid(row=group,column=0, padx=10,ipady=4, sticky=tk.W)
        

In [5]:

class GetValues:
    def __init__(self,w, data, method) :
        
        # get values of entries
        self.w= w
        self.AllSub= data.AllSub
        self.method = method
        # CrossValue= [data.SelectedClass,data.SelectedOnset, 
        #                     data.TrainSub,data.ValSub ,data.TestSub,
        #                     data.TrainSess,data.ValSess, data.TestSess,
        #                     data.TrainSplit, data.ValSplit]
        
        
        data.SelectedClass= self.CheckEmptyValues(data.SelectedClass)
        data.SelectedOnset= self.CheckEmptyValues(data.SelectedOnset)
        data.TrainSub= self.CheckEmptyValues(data.TrainSub)
        data.ValSub= self.CheckEmptyValues(data.ValSub)
        data.TestSub= self.CheckEmptyValues(data.TestSub)
        data.TrainSess= self.CheckEmptyValues(data.TrainSess)
        data.ValSess= self.CheckEmptyValues(data.ValSess)
        data.TestSess= self.CheckEmptyValues(data.TestSess)

        if self.method== 1: #5fold
            data.TrainSplit = 0.8
        elif self.method ==2: #10fold
            data.TrainSplit = 0.9
        elif self.method == 4:
            data.TrainSplit = self.CheckEmptyValues(data.TrainSplit)
        #self.w.destroy()
                
        
    def CheckEmptyValues(self, value):

        #check empty values
        if len(value.get()) != 0:
                temp= str(value.get())
                value= self.CompleteNumber(temp)
                
        else:
                messagebox.showerror("Error",f'You must enter a number.', parent=self.w)
        
        # check input values that are not in databse
        # need to be revised 
        return value

    def CompleteNumber(self,string): # for 1~5 -> 1,2,3,4,5
        if '~' in string: 
            string= string.split('~')
            strList= [sub for sub in self.AllSub if sub >= string[0] and sub <= string[1]]

        elif ',' in string:
            strList= string.split(',')

        else:
            strList= string
            
        return strList
        


In [6]:
class SplitData:
    def __init__(self, groupw,data):
        self.groupw= groupw
        self.AllSub= data.AllSub
        self.TrainSub= data.TrainSub
        self.q= self.SubOrderInit()
        
        # if len(data.TrainSub) == 1:
        #     self.Individual()
        # else:
        self.CrossSubject()
        
    def SubOrderInit(self):
        q= queue.Queue()
        [q.put(j) for j in self.AllSub]
        while ( q.queue[0] != self.TrainSub[0]):
            temp=q.get()
            q.put(temp) # [3,4,5,6,1,2] the first element is the selected sub
        return q

    # def Individual(self):
    #     for sub in self.q.queue:
    #         #trainloader
    #         print(sub)

    
        
    def CrossSubject(self):
       
        for group in range(len(self.AllSub)):
            batchsub=list(self.q.queue)[0:len(self.TrainSub)]
            print(group, batchsub) 

            
            temp= self.q.get() # get first sub
            self.q.put(temp)   # put to the tail

    
            #training process 
            # aftersub.get() # kick out one subject 
           # 前四個train 最後一個test 
                
            #tk.Label(self.w , text= self.q.queue, bg= 'White').grid(row=group,column=0, padx=10,ipady=4, sticky=tk.W)
        # aftersub= queue.Queue()
        # for group in range(len(self.AllSub)):
        #     # trainloader = SelectedSubtrain
    
        #     while  (len(aftersub.queue) <= len(self.TrainSub)*2-1) :
        #         temp= self.q.get()
        #         aftersub.put(temp) 
        #         self.q.put(temp)
        #         # if len(q.queue) < len()
        #        # 1 3 4 5 7 8 
        #        # 
            
        #     #training scheme
        #     for i in range(len(self.TrainSub)):
        #         aftersub.get()
        #     tk.Label( self.groupw , text= aftersub.queue, bg= 'White').grid(row=group,column=0, padx=10,ipady=4, sticky=tk.W)
        #     print(aftersub.queue)

# 2 , 4-1 
# 3, 6-2
# 4, 9-3  
# 5  10-5       


In [7]:
from database import Database

w = tk.Tk()
w.geometry('400x650')
w.title("ExBrainable")
w.configure(background='White')

d= Database()
CrossValidation_Panel(w, d)
w.mainloop()

In [23]:
Data =[]
Label=[]

AllSub=[1,2,3,4,5,6]

SelectedSession= [1] #session x change
Split= [] #split x change
ExcludeClass=[1] #class x change
Onsetime=[] #onset 
SelectedSubtrain=[3] # SubOrderInit()




#initialization
def SubOrderInit(AllSub, SelectedSubtrain):
    q= queue.Queue()
    [q.put(j) for j in AllSub]
    while ( q.queue[0] != SelectedSubtrain[0]):
        temp=q.get()
        q.put(temp) # [3,4,5,6,1,2] the first element is the selected sub
    return q


# training , individdual 
q = SubOrderInit(AllSub, SelectedSubtrain)
for sub in q.queue:
    #trainloader
    print(sub)


# cross subject
SelectedSubtrain= [1,2,3,4]
SelectedSubval= [5,6,7]
SelectedSubtest =[8,9]

q = SubOrderInit(AllSub, SelectedSubtrain)
aftersub= queue.Queue()
for group in range(len(AllSub)):
    # trainloader = SelectedSubtrain
    while  len(aftersub.queue) <= len(SelectedSubtrain):
        temp= q.get()
        aftersub.put(temp) #[]
        q.put(temp)

    #training scheme
    aftersub.get()
    
    print(aftersub.queue)




3
4
5
6
1
2
deque([2, 3, 4, 5])
deque([3, 4, 5, 6])
deque([4, 5, 6, 1])
deque([5, 6, 1, 2])
deque([6, 1, 2, 3])
deque([1, 2, 3, 4])


In [8]:
path= 'G:/我的雲端硬碟/109_2 course/Project/ExBrainable/GUI/xydata/S03/train_epoch.set'
raw = mne.io.read_epochs_eeglab(path, uint16_codec='latin1') 
path= 'G:/我的雲端硬碟/109_2 course/Project/ExBrainable/GUI/xydata/S03/test_epoch.set'
raw_t = mne.io.read_epochs_eeglab(path, uint16_codec='latin1') 

Extracting parameters from G:/我的雲端硬碟/109_2 course/Project/ExBrainable/GUI/xydata/S03/train_epoch.set...
Not setting metadata
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Ready.
Extracting parameters from G:/我的雲端硬碟/109_2 course/Project/ExBrainable/GUI/xydata/S03/test_epoch.set...
Not setting metadata
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Ready.


In [9]:
Data= {}
Data['data_1']= raw
Data['data_2']= raw_t

In [10]:
def MakeInfo(raw, sub,session):
    onset= np.diff(raw.events[:,0])
    # insert length of last trial, make sure order is (trial, ch, tp)
    onset= np.insert(onset, len(raw.events)-1 , raw._data[-1,-1,:].shape , axis=0) 
    data={ 'Subject': [sub]* len(raw.events),
           'Session': session, 
        'Class': raw.events[:,2],
        'Onset time':  onset}
    df= pd.DataFrame(data)

    return df

i=0
for file in Data.keys():
    sub=[3,4] #revise
    session= [1,2]
    df=MakeInfo(Data[file], sub[i], session[i])
    i+=1
    if 'Info' not in globals():
        Info= df
    else:
        temp= df
        Info= pd.concat([Info, temp])
    
 # event_id, selection, events





In [11]:
# schedule for all subjects
ExcludeClass=[2]
Onsetime=[]

SelectedSession= [1]
SelectedSub=[3]


mask= Info['Class'].isin(ExcludeClass)
SelectedData= Info[~ mask]

include= SelectedData['Session'].isin(SelectedSession)
SelectedData= SelectedData[include]

include= SelectedData['Subject'].isin(SelectedSub)
SelectedData= SelectedData[include]
print(SelectedData)



     Subject  Session  Class  Onset time
0          3        1      1         562
3          3        1      1         562
5          3        1      1         562
7          3        1      3         562
9          3        1      4         562
..       ...      ...    ...         ...
283        3        1      3         562
284        3        1      4         562
285        3        1      3         562
286        3        1      1         562
287        3        1      3         562

[216 rows x 4 columns]


In [10]:
df= SelectedData.set_index(keys=['Subject', SelectedData.index])

In [15]:
Data['data_1'][SelectedData.index]

array([[[ 5.37036133e-06,  7.99856806e-06,  3.57174087e-06, ...,
          4.79417849e-06,  5.10002184e-06,  1.05693269e-06],
        [ 2.17053145e-07,  2.67895007e-06, -1.16001308e-06, ...,
          6.46865892e-06,  7.20865011e-06,  5.42524695e-07],
        [ 2.19902921e-06,  4.45230436e-06,  3.55793089e-07, ...,
          7.35926676e-06,  5.75072145e-06,  1.05064251e-07],
        ...,
        [ 5.73590755e-06,  7.18024111e-06,  9.08615410e-07, ...,
          7.64622879e-06,  5.59462023e-06, -4.31652963e-07],
        [ 4.52549028e-06,  6.13829041e-06,  9.26241279e-07, ...,
          8.97784233e-06,  6.92971659e-06,  4.91959810e-07],
        [ 6.47993898e-06,  7.94386005e-06,  1.16139829e-06, ...,
          8.35280609e-06,  6.39465332e-06,  6.32639825e-07]],

       [[-5.28079557e-06, -8.17343330e-06, -3.60113645e-06, ...,
          5.77581644e-06, -3.24440956e-06, -5.14074183e-06],
        [-1.09929342e-05, -8.79222965e-06,  1.17805719e-07, ...,
          2.18263197e-06, -7.16359997e

In [61]:
df.index.values

array([(3, 1), (3, 2), (3, 4), (3, 6), (3, 7), (3, 8), (3, 9), (3, 11),
       (3, 12), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 20),
       (3, 21), (3, 24), (3, 25), (3, 27), (3, 28), (3, 29), (3, 31),
       (3, 33), (3, 34), (3, 36), (3, 37), (3, 38), (3, 39), (3, 40),
       (3, 41), (3, 42), (3, 43), (3, 44), (3, 45), (3, 46), (3, 47),
       (3, 48), (3, 49), (3, 50), (3, 53), (3, 54), (3, 55), (3, 56),
       (3, 57), (3, 61), (3, 62), (3, 63), (3, 66), (3, 68), (3, 69),
       (3, 70), (3, 71), (3, 73), (3, 74), (3, 75), (3, 76), (3, 77),
       (3, 78), (3, 80), (3, 81), (3, 82), (3, 83), (3, 84), (3, 85),
       (3, 87), (3, 88), (3, 89), (3, 90), (3, 91), (3, 92), (3, 93),
       (3, 94), (3, 98), (3, 99), (3, 101), (3, 103), (3, 104), (3, 105),
       (3, 107), (3, 108), (3, 110), (3, 111), (3, 112), (3, 113),
       (3, 114), (3, 115), (3, 117), (3, 118), (3, 119), (3, 120),
       (3, 122), (3, 123), (3, 125), (3, 126), (3, 128), (3, 129),
       (3, 130), (3, 13

In [60]:
SelectedData.to_numpy()

array([[  3,   1,   2, 562],
       [  3,   1,   2, 562],
       [  3,   1,   2, 562],
       [  3,   1,   2, 562],
       [  3,   1,   3, 562],
       [  3,   1,   2, 562],
       [  3,   1,   4, 562],
       [  3,   1,   3, 562],
       [  3,   1,   2, 562],
       [  3,   1,   4, 562],
       [  3,   1,   4, 562],
       [  3,   1,   4, 562],
       [  3,   1,   4, 562],
       [  3,   1,   4, 562],
       [  3,   1,   3, 562],
       [  3,   1,   2, 562],
       [  3,   1,   3, 562],
       [  3,   1,   4, 562],
       [  3,   1,   3, 562],
       [  3,   1,   3, 562],
       [  3,   1,   3, 562],
       [  3,   1,   2, 562],
       [  3,   1,   2, 562],
       [  3,   1,   2, 562],
       [  3,   1,   2, 562],
       [  3,   1,   3, 562],
       [  3,   1,   2, 562],
       [  3,   1,   3, 562],
       [  3,   1,   3, 562],
       [  3,   1,   4, 562],
       [  3,   1,   3, 562],
       [  3,   1,   3, 562],
       [  3,   1,   4, 562],
       [  3,   1,   4, 562],
       [  3,  